### Roman Imaging WCS

#### Generalized World Coordinate System (GWCS) Overview

Roman and JWST use the [GWCS](https://gwcs.readthedocs.io/en/latest/) for managing the World Coordinate System WCS). We call "WCS" the mapping from "pixel" coordinates to some "real-world" physical coordinates - celestial, spectral, time, etc. GWCS is a generalized implementation of WCS aiming to avoid the limitations of the FITS WCS standard. It is a flexible toolkit for expressing and evaluating transformations between pixel and world coordinates, as well as intermediate coordinates. The GWCS object supports a data model which includes the entire transformation pipeline from input pixel coordinates to world coordinates (and vice versa). 

GWCS is based on astropy and supports the [Shared Interface for WCS](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst). The WCS "pipeline" is a list of steps, where each step is a tuple of coordinate frame and a transform to the next frame. The transform in the last step is `None` representing the final coordinate frame of the WCS pipeline.

Transforms are based on [astropy.modeling](https://docs.astropy.org/en/stable/modeling/) and include support for [coordinate units](https://docs.astropy.org/en/stable/units/index.html). Coordinate frames utilize [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/index.html). 
The GWCS object is serialized to [ASDF](https://asdf-standard.readthedocs.io/en/latest/) using the ASDF [WCS](https://asdf-wcs-schemas.readthedocs.io/en/latest/) and [transforms](https://asdf-transform-schemas.readthedocs.io/en/latest/) extensions.

Download a Roman Level2 simulated file. The downloaded file is called `r002_assign_wcs.asdf`.

#### Roman Imaging WCS

The `assign_wcs` step in the Roman calibration pipeline constructs a WCS object and assigns it to the Level 2 image. The distortion transformations are stored in a reference file in CRDS, with `reftype=distortion`. The distortion includes all transformsations from a detector to a coordinate frame associated with the telescope `(V2, V3)`. The telescope telemetry is used to transform from `(V2, V3)` to celestial coordinates.



We can use `asdf` or `roman_datamodels` to open the file and retrieve the WCS object.

In [ ]:
from roman_datamodels import datamodels as rdm

In [ ]:
image = rdm.open('../data/r0000101001001001001_0001_wfi01_cal.asdf')
image.search('wcs')

The above `search` commands finds the string `wcs` in three places. Using `info` shows that `wcsinfo` is a dictionary holding the pointing `(ra_ref, dec_ref)`, the reference point of the aperture in the telescope `(V2, V3)` system and other WCS related parameters.

In [ ]:
image.info(max_rows=600, max_cols=600,show_values=True)

The `WCS` object is in the `image.meta.wcs` attribute.

In [ ]:
w = image.meta.wcs

`print` shows the WCS pipeline

In [ ]:
print(w)

`repr` prints all transforms. Notable below is the `Expression` section which shows how the transforms are combined. The "join" operator `&` combines two models which take independent inputs and concatenates the outputs. The "conposition" operator, `|`, combines models in series, i.e. chains the output of the first one as input to the second. More information on combining models is available at [Compound Models](https://docs.astropy.org/en/stable/modeling/compound-models.html#).

In [ ]:
w

In [ ]:
from matplotlib import pyplot as plt
#%matplotlib notebook
#%maptlotlib ipympl
from astropy import units as u

In [ ]:
ax = plt.subplot(projection=image.meta.wcs)
plt.imshow(image.data, vmin=.1, vmax=.4, origin='lower')
plt.gray()
plt.grid(color='white', ls='solid')
ax.coords[0].set_format_unit(u.degree)
ax.coords[0].coord_wrap = 180 * u.deg

The WCS can be evaluated directly as a function or using the Shared WCS Interface methods

In [ ]:
from gwcs.wcstools import grid_from_bounding_box

The `bounding_box` of a WCS object represents the range of input values over which the transforms are valid. Typically it is the full detector. By default, input values outside the `bounding box` return `NaN`s. 

In [ ]:
w.bounding_box

In [ ]:
x, y = grid_from_bounding_box(w.bounding_box)

Evaluating the WCS object returns the numerical values of the result.

In [ ]:
ra, dec = w(x, y)
print (ra, dec)

Using the Shared WCS interface methods returns a Python object of type astropy.coordinates.SkyCoord.

In [ ]:
sky = w.pixel_to_world(x, y)

In [ ]:
print(sky)

Other useful methods include

`getting a transform between intermediate frames`

In [ ]:
detector_to_v2v3 = w.get_transform('detector', 'v2v3')
print(detector_to_v2v3)

`showing the coorindate frames in the WCS pipeline`

In [ ]:
print(w.available_frames)

Retrieving the entire `forward` or `backward` transform

In [ ]:
print(w.forward_transform)

In [ ]:
print(w.backward_transform)

Other GWCS features include the ability to

- insert additional frames and transforms
- compute the inverse of the transforms using an iterative numerical method (in case an inverse transform is not provided)
- create  FITS approximation using the SIP convention to represent the distortion
- convert to other celestial frames usnig the `Shared API`
